# 주의사항

- 엥간하면 한 번에 끝낼 것

# gpu 세팅 후 확인

In [1]:
!nvidia-smi

Sun Feb  8 05:28:00 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Claude code가 현재 노트북을 읽나 체크

네, 노트북을 읽을 수 있습니다! 

현재 colab_cc_connection.ipynb에는 다음 내용이 있습니다:

1. **GPU 세팅 확인** - NVIDIA Tesla T4 GPU가 설정되어 있고, 15360MiB 메모리를 가진 GPU가 인식되고 있습니다
2. **드라이브 마운트** - Google Colab에서 드라이브를 마운트하는 코드
3. **마지막 셀** - "Claude code가 현재 노트북을 읽나 체크" 라는 마크다운 헤더가 있습니다

제가 정상적으로 노트북을 읽고 있으니, 이제 작업을 진행하실 수 있습니다!

# 드라이브 마운트

In [4]:
from google.colab import drive
#drive.mount('/content/drive')

아직 코멘트처리한 부분은 되지 아니함>>이었다가 2026/2/8 기준 성공

In [5]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 마운트한 내용 읽기

In [19]:
!ls

drive  sample_data


In [14]:
import pandas as pd

In [23]:
df = pd.read_csv(r'/content/drive/MyDrive/Euro Current/EUR_KRW.csv')

단, 마운트를 해도 파일 위치를 모르면 어렵다.

# 읽은 내용 기반으로 cc 돌려보기

In [24]:
print(df.shape)
print(df.dtypes)
df.head()

(783, 7)
날짜      object
종가      object
시가      object
고가      object
저가      object
거래량     object
변동 %    object
dtype: object


,날짜,종가,시가,고가,저가,거래량,변동 %
0,2023- 06- 12,"1,384.13","1,383.67","1,390.35","1,383.67",NaN,0.03%
1,2023- 06- 11,"1,383.67","1,383.67","1,383.67","1,383.67",NaN,0.01%
2,2023- 06- 09,"1,383.48","1,398.03","1,400.39","1,382.69",NaN,-0.94%
3,2023- 06- 08,"1,396.65","1,400.13","1,402.52","1,393.85",NaN,-0.20%
4,2023- 06- 07,"1,399.45","1,389.61","1,400.96","1,387.30",NaN,0.74%


In [25]:
# EUR_KRW 전처리 후 기본 통계
df_eur = pd.read_csv(r'/content/drive/MyDrive/Euro Current/EUR_KRW.csv')

# 타입 변환 (쉼표, %, 공백 제거)
df_eur['날짜'] = pd.to_datetime(df_eur['날짜'].str.replace(' ', ''))
for col in ['종가', '시가', '고가', '저가']:
    df_eur[col] = df_eur[col].str.replace(',', '').astype(float)
df_eur['변동 %'] = df_eur['변동 %'].str.replace('%', '').astype(float)
df_eur = df_eur.drop(columns=['거래량']).sort_values('날짜').reset_index(drop=True)

print(f"기간: {df_eur['날짜'].min().date()} ~ {df_eur['날짜'].max().date()}")
print(f"총 거래일수: {len(df_eur)}일\n")
print(df_eur.describe())

기간: 2020-06-12 ~ 2023-06-12
총 거래일수: 783일

                                  날짜           종가           시가           고가  \
count                            783   783.000000   783.000000   783.000000   
mean   2021-12-12 04:45:03.448275712  1362.319323  1363.020345  1369.218787   
min              2020-06-12 00:00:00  1310.040000  1310.290000  1314.030000   
25%              2021-03-13 12:00:00  1340.500000  1341.085000  1346.565000   
50%              2021-12-13 00:00:00  1353.840000  1354.110000  1360.410000   
75%              2022-09-12 12:00:00  1378.615000  1379.000000  1387.380000   
max              2023-06-12 00:00:00  1477.160000  1477.900000  1484.860000   
std                              NaN    31.372408    31.330635    31.632861   

                저가        변동 %  
count   783.000000  783.000000  
mean   1357.345364    0.003193  
min    1303.690000   -1.440000  
25%    1335.435000   -0.300000  
50%    1349.320000   -0.010000  
75%    1373.880000    0.315000  
max    1475.190

## 결론

- 드라이브 마운트는 되나 드라이브 속 경로를 정확히 입력해야 함.
- claude code는 코드를 노트에 입력시켜주나 실행은 내 몫. 실행이 자동으로 안 되는 점이 미스.
- 실행과 경로가 장애물. 해결책 모색해보기

## 로컬 속 내용 기반으로 cc 돌려보기

In [4]:
import pandas as pd

In [ ]:
pd.read_csv('c:\Users\Choi\Downloads\input_re.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\Choi\\Downloads\\input_re.csv'

문제점: Colab은 Google 서버에서 실행되기 때문에 로컬 Windows 경로(c:\Users\...)에 직접 접근할 수 없습니다. 

## 구글 드라이브 속 파일의 링크를 활용해보자

In [16]:
pd.read_csv('https://drive.google.com/file/d/1rGvduNtdKpgCnR8vjzNsPqm2Li5tQKZf/view?usp=drive_link')

HTTPError: HTTP Error 401: Unauthorized

2026 1월 18일 기준으로 vscode 상 colab에는 데이터를 업로드하기 매우 어려워서 사용이 어려움

In [28]:
import pandas as pd

In [29]:
df = pd.read_csv('/content/sample_data/mnist_train_small.csv', header=None)
print(f"데이터 shape: {df.shape}")
df.head()

데이터 shape: (20000, 785)


,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Simple MLP Model for MNIST

In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np

In [31]:
# 데이터 준비
X = df.iloc[:, 1:].values.astype(np.float32) / 255.0  # 픽셀값 정규화
y = df.iloc[:, 0].values.astype(np.int64)  # 첫 번째 열이 레이블

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tensor 변환
X_train = torch.tensor(X_train)
X_test = torch.tensor(X_test)
y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)

# DataLoader 생성
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print(f"Train: {len(X_train)}, Test: {len(X_test)}")

Train: 16000, Test: 4000


In [32]:
# MLP 모델 정의
class MLP(nn.Module):
    def __init__(self, input_size=784, hidden_size=128, num_classes=10):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_size, num_classes)
        )
    
    def forward(self, x):
        return self.layers(x)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MLP().to(device)
print(f"Using device: {device}")
print(model)

Using device: cuda
MLP(
  (layers): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=128, out_features=128, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [33]:
# 학습 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss/len(train_loader):.4f}")

Epoch [1/10], Loss: 0.7003
Epoch [2/10], Loss: 0.2981
Epoch [3/10], Loss: 0.2251
Epoch [4/10], Loss: 0.1820
Epoch [5/10], Loss: 0.1501
Epoch [6/10], Loss: 0.1235
Epoch [7/10], Loss: 0.1091
Epoch [8/10], Loss: 0.0963
Epoch [9/10], Loss: 0.0872
Epoch [10/10], Loss: 0.0795


In [34]:
# 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        _, predicted = torch.max(outputs.data, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 96.10%


content 안에 거 밖에 못함. 마운트 올해 안에 되면 좋겠다

파일만 올라간다면 claude code를 쓸 수 있음